# Finetuning anomalib model

1. Import libraries and prepare datamodule

In [21]:
from anomalib.data.image.folder import Folder
from anomalib.models import Cfa
from anomalib.engine import Engine
import torch

# Initialize datamodule with reduced number of workers
datamodule = Folder(
    name="dataset",
    root="dataset",
    normal_dir="Sans_Defaut",
    abnormal_dir="Defaut",
    normal_split_ratio=0.2,  # 20% of normal for validation/testing
    val_split_mode="from_test",
    val_split_ratio=0.5,
    image_size=(256, 256),
    task="classification",
    num_workers=2  # Reduced number of workers
)

2. Configure model and engine

In [22]:
# Model configuration with reduced coreset sampling ratio and smaller backbone
model = Cfa(
    backbone="resnet18"
    )

# Training setup with mixed precision
engine = Engine(
    max_epochs=50,
    devices=1,
    precision="16-mixed",  # Use mixed precision to reduce memory usage
    pixel_metrics=["AUROC", "F1Score"],
    task="classification",
)

INFO:anomalib.models.components.base.anomaly_module:Initializing Cfa model.
/home/joey/M1/ouverture_recherche_2024/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/joey/M1/ouverture_recherche_2024/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


3. Train and save the model

In [23]:
# Start training
engine.fit(datamodule=datamodule, model=model)

# Save the model
torch.save(model.state_dict(), 'model.pth')

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
INFO:anomalib.data.base.datamodule:No normal test images found. Sampling from training set using a split ratio of 0.20
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type                     | Params | Mode 
---------------------------------------------------------------------------
0 | model                 | CfaModel                 | 3.2 M  | train
1 | loss                  | CfaLoss                  | 0      | train
2 | _transform            | Compose                  | 0      | train
3 | normalization_metrics | MetricCollection         | 0      | train
4 | image_threshold       | F1AdaptiveThreshold      | 0      | train
5 | pixel_threshold       | F1AdaptiveThreshold      | 0      | train
6 | image_metrics         | AnomalibMetricCollection | 0      | train
7 | pixel_metrics         | AnomalibMetric

Training: |          | 0/? [00:00<?, ?it/s]

  0%|          | 0/51 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 2.93 GiB of which 127.38 MiB is free. Including non-PyTorch memory, this process has 1.67 GiB memory in use. Of the allocated memory 1.39 GiB is allocated by PyTorch, and 221.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)